In [1]:
import numpy as np

from mnist import MNIST
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

N_CLASSES = 10

print("Loading dataset...")
mndata = MNIST("./data/")
images, labels = mndata.load_training()
images = np.array(images).reshape(-1,28,28)
print("Done")

Loading dataset...
Done


In [2]:
def getSlides(X, windowSize):
    X_slides = []
    for i in range(X.shape[1]-windowSize+1):
        for j in range(X.shape[2]-windowSize+1):
            X_slides.append(X[:,i:i+10,j:j+10])
            
    n_slides = (X.shape[1]-windowSize+1)*(X.shape[2]-windowSize+1)
    new_shape = ( n_slides, X.shape[0], windowSize**2 )     
    X_slides = np.array(X_slides).reshape(new_shape)  # (slides, samples, 10x10)
    return X_slides.reshape( (n_slides*X.shape[0], windowSize**2), order='F'), n_slides #(samplesXslides, 28x28)

def getLabels(y, numSlides):
    return np.array([y for n in range(numSlides)]).reshape(-1, order='F')

In [3]:
class MultiGrainedScanning:
    def __init__(self, windowSize, clfs):
        self.clfs = clfs # list of classifiers
        self.windowSize = windowSize
    
    def train(self, X, y):
        X, n_slides = getSlides(X, self.windowSize)
        y = getLabels(y, n_slides)
        [clf.fit(X, y) for clf in self.clfs]
        
    def predict_proba(self, X):
        X, n_slides = getSlides(X, self.windowSize)
        re-=sults = [clf.predict_proba(X).reshape(-1, n_slides, N_CLASSES) for clf in self.clfs]
        return np.concatenate(results, axis=1)

In [4]:
clf1 = RandomForestClassifier(n_estimators=500, max_leaf_nodes = 100, n_jobs=6)
clf2 = RandomForestClassifier(n_estimators=500, max_leaf_nodes = 100, n_jobs=6)
mgs = MultiGrainedScanning(10, [clf1, clf2])
# Train on the first 10000 images:
train_x = images[:10000]
train_y = labels[:10000]

mgs.train(train_x, train_y)

test_x = images[-1000:]
test_y = labels[-1000:]
predictions = mgs.predict_proba(test_x)
predictions.shape

(1000, 722, 10)

In [5]:
ave = np.average(predictions, axis = 1)
pred = np.argmax(ave, axis=1)
acc = np.sum(pred==test_y)/len(pred)
print("accuracy:", acc)

accuracy: 0.773
